# VIEW DATA
This notebook view and inspect raw data from January


In [1]:
import pandas as pd

## VIEW January's raw data


In [11]:
df1 = pd.read_parquet("../raw/yellow_tripdata_2021-01.parquet")
print("Raw data from January has shape:", df1.shape)


Raw data from January has shape: (1369769, 19)


In [ ]:
print("First ten rows of the data: ")
df1.head(10)

In [ ]:
print("General information about columns, non-null count, and type of data of a month data:")
df1.info()

#### Findings from `.info()`

* **Dtypes:** Data types are mostly correct. `tpep_pickup_datetime` and `tpep_dropoff_datetime` are already proper `datetime64[ns]` objects.
* **Null Values:**
    * `airport_fee`: Almost 100% null (only 5 non-null values)
    * `passenger_count`, `RatecodeID`, `store_and_fwd_flag`, `congestion_surcharge`: These columns share the same number of missing values (~98k, or 7% of the data). 

-> Action: This needs to be handled in the QA step.

In [ ]:
print("Info about numerical columns in data:")
df1.describe()

#### Findings from `.describe()`

This reveals several anomalies and outliers that will define our QA Rules:
1.  **`passenger_count`**: `min = 0.0`. (An invalid value -> Rule: Must be > 0).
2.  **`trip_distance`**: `min = 0.0` and `max` is extremely large (263k). (Invalid values -> Rule: Must be > 0 and below a reasonable threshold).
3.  **`fare_amount`**: `min = -490.0`. (Invalid value -> Rule: Must be > 0).
4.  **`total_amount`**: `min = -492.8`. (Invalid value -> Rule: Must be > 0).

In [ ]:
print("Columns of a month data are:")
df1.columns

## VIEW DATA TABLE ZONE

In [3]:
tzl = pd.read_csv("../raw/taxi_zone_lookup.csv")
print(tzl.shape)
tzl.head(5)

(265, 4)


,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [ ]:
tzl.describe()

In [ ]:
tzl.info()

In [4]:
tzl['Borough'].unique()

array(['EWR', 'Queens', 'Bronx', 'Manhattan', 'Staten Island', 'Brooklyn',
       'Unknown', nan], dtype=object)

In [ ]:
tzl['Zone'].describe()

count                                               264
unique                                              261
top       Governor's Island/Ellis Island/Liberty Island
freq                                                  3
Name: Zone, dtype: object

In [9]:
tzl['service_zone'].describe()

count           263
unique            4
top       Boro Zone
freq            205
Name: service_zone, dtype: object

## VIEW DATA OF SOME SPECIFIC COLUMNS


In [13]:
df1.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [ ]:
print("Values in column VendorID are: ")
df1['VendorID'].value_counts()

In [ ]:
print("Values in column Payment type are:")
df1['payment_type'].value_counts()

In [ ]:
print("Statistic info of Passenger count column is: ")
df1['passenger_count'].describe()

In [ ]:
print("Statistic info of Extra column is: ")
df1['extra'].describe()

In [ ]:
print("Unique values in Airport fee column are:")
list(df1['airport_fee'].unique())

In [ ]:
print("The number of non-null values and null values(NA) in column Store and fwd flag is:")
df1['store_and_fwd_flag'].notna().value_counts()

In [16]:
df1['congestion_surcharge'].describe()

count    1.271417e+06
mean     2.239047e+00
std      7.989435e-01
min     -2.500000e+00
25%      2.500000e+00
50%      2.500000e+00
75%      2.500000e+00
max      3.000000e+00
Name: congestion_surcharge, dtype: float64